In [ ]:
!nvidia-smi

In [ ]:
import gc
import csv
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.ndimage import gaussian_filter
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GroupKFold
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning import seed_everything
from torchmetrics.image import StructuralSimilarityIndexMeasure as SSIM

In [ ]:
class Config:
    debug = False
    use_l4_gpu = False
    seed = 42
    n_folds = 5
    epochs = 100
    train_batch_size = 64
    val_batch_size = 64
    test_batch_size = 8
    num_workers = 4
    es_patience = 20

In [ ]:
if Config.use_l4_gpu:
    torch.set_float32_matmul_precision('high')

In [ ]:
seed_everything(Config.seed, workers=True)

In [ ]:
all_inputs = [
    f for f in Path('../input/waveform-inversion/train_samples').rglob('*.npy')
    if ('seis' in f.stem) or ('data' in f.stem)
]

def inputs_files_to_output_files(input_files):
    return [Path(str(f).replace('seis', 'vel').replace('data', 'model')) for f in input_files]

all_outputs = inputs_files_to_output_files(all_inputs)
assert all(f.exists() for f in all_outputs)

test_files = list(Path('../input/waveform-inversion/test').glob('*.npy'))

In [ ]:
def apply_gaussian_filter(data, sigma=(1, 1)):
    filtered = np.zeros_like(data)
    for c in range(data.shape[0]):
        filtered[c] = gaussian_filter(data[c], sigma=sigma)
    return filtered

In [ ]:
class SeismicDataset(Dataset):
    def __init__(self, inputs_files, output_files, n_examples_per_file=500, augment=False):
        assert len(inputs_files) == len(output_files)
        self.inputs_files = inputs_files
        self.output_files = output_files
        self.n_examples_per_file = n_examples_per_file
        self.augment = augment

    def __len__(self):
        return len(self.inputs_files) * self.n_examples_per_file

    def _apply_augmentation(self, x, y):
        if np.random.rand() < 0.5:
            x = x.copy()
            y = y.copy()
            noise = np.random.normal(0, np.random.uniform(0.001, 0.01), size=x.shape).astype(np.float32)
            x += noise

        if np.random.rand() < 0.5:
            x = np.flip(x, axis=2).copy()
            y = np.flip(y, axis=2 if y.ndim == 3 else 1).copy()

        return x, y

    def __getitem__(self, idx):
        file_idx = idx // self.n_examples_per_file
        sample_idx = idx % self.n_examples_per_file

        X = np.load(self.inputs_files[file_idx], mmap_mode='r')
        y = np.load(self.output_files[file_idx], mmap_mode='r')

        x_sample = X[sample_idx].astype(np.float32).copy()
        y_sample = y[sample_idx].astype(np.float32).copy()

        if self.augment:
            x_sample, y_sample = self._apply_augmentation(x_sample, y_sample)

        x_sample = apply_gaussian_filter(x_sample)

        return torch.from_numpy(x_sample), torch.from_numpy(y_sample)

In [ ]:
class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.84, beta=0.16):
        super().__init__()
        self.l1 = nn.L1Loss()
        self.ssim = SSIM(data_range=1.0)
        self.alpha = alpha
        self.beta = beta

    def forward(self, pred, target):
        pred = (pred - 1500) / 1000
        target = (target - 1500) / 1000

        l1_loss = self.l1(pred, target)
        ssim_loss = 1 - self.ssim(pred, target)

        return self.alpha * l1_loss + self.beta * ssim_loss

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, use_se=True, dropout_prob=0.1):
        super().__init__()
        self.use_se = use_se
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout2d(dropout_prob) if dropout_prob > 0 else nn.Identity()

        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.downsample = None

        if self.use_se:
            self.se = nn.Sequential(
                nn.AdaptiveAvgPool2d(1),
                nn.Conv2d(out_channels, out_channels // 16, kernel_size=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels // 16, out_channels, kernel_size=1),
                nn.Sigmoid()
            )

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.dropout(out)

        if self.use_se:
            out = out * self.se(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        return self.relu(out)


class SeismicModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()

        self.stem = nn.Sequential(
            nn.Conv2d(5, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )

        self.layer1 = nn.Sequential(
            ResidualBlock(32, 64, stride=2, use_se=True, dropout_prob=0.1),
            ResidualBlock(64, 64, stride=1, use_se=True, dropout_prob=0.1)
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128, stride=2, use_se=True, dropout_prob=0.1),
            ResidualBlock(128, 128, stride=1, use_se=True, dropout_prob=0.1)
        )
        self.layer3 = nn.Sequential(
            ResidualBlock(128, 256, stride=2, use_se=True, dropout_prob=0.1),
            ResidualBlock(256, 256, stride=1, use_se=True, dropout_prob=0.1)
        )
        self.layer4 = nn.Sequential(
            ResidualBlock(256, 512, stride=2, use_se=True, dropout_prob=0.1),
            ResidualBlock(512, 512, stride=1, use_se=True, dropout_prob=0.1)
        )
        self.layer5 = nn.Sequential(
            ResidualBlock(512, 1024, stride=2, use_se=True, dropout_prob=0.1),
            ResidualBlock(1024, 1024, stride=1, use_se=True, dropout_prob=0.1)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1),
        )

        self.loss_fn = CombinedLoss()

    def forward(self, x):
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.decoder(x)
        x = F.interpolate(x, size=(70, 70), mode='bilinear', align_corners=False)
        return x * 1000 + 1500

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.loss_fn(preds, y)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.loss_fn(preds, y)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        return loss

    def predict_step(self, batch, batch_idx):
        x, file_id = batch
        return self(x)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters())
        scheduler = {
            'scheduler': ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True),
            'monitor': 'val_loss',
            'interval': 'epoch',
            'frequency': 1
        }
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

In [ ]:
def run_train(fold, all_inputs, all_outputs, tr_idx, val_idx, config):
    print(f'     ----- Fold: {fold} -----')
    train_inputs = [all_inputs[i] for i in tr_idx]
    val_inputs = [all_inputs[i] for i in val_idx]
    if config.debug:
        train_inputs = [train_inputs[0]]
        val_inputs = [val_inputs[0]]
    train_outputs = inputs_files_to_output_files(train_inputs)
    val_outputs = inputs_files_to_output_files(val_inputs)

    dstrain = SeismicDataset(train_inputs, train_outputs, augment=True)
    dsvalid = SeismicDataset(val_inputs, val_outputs)

    dltrain = DataLoader(dstrain,
                         batch_size=config.train_batch_size,
                         shuffle=True,
                         num_workers=config.num_workers,
                         pin_memory=True,
                         drop_last=True,
                         persistent_workers=True)

    dlvalid = DataLoader(dsvalid,
                         batch_size=config.val_batch_size,
                         shuffle=False,
                         num_workers=config.num_workers,
                         pin_memory=True)

    model = SeismicModel()

    checkpoint = ModelCheckpoint(monitor='val_loss',
                                 mode='min',
                                 save_top_k=1,
                                 save_weights_only=True,
                                 dirpath=f'model_fold{fold}/')
    es_callback = EarlyStopping(monitor='val_loss',
                                patience=config.es_patience)
    logger = CSVLogger(save_dir='logs/', name=f'model_fold{fold}')

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    trainer = Trainer(
        max_epochs=config.epochs,
        accelerator='auto',
        logger=logger,
        callbacks=[checkpoint,es_callback])

    trainer.fit(model, dltrain, dlvalid)

    model.load_state_dict(torch.load(checkpoint.best_model_path)['state_dict'])
    model.eval()

    mae_losses = []

    for batch in dlvalid:
        x, y = batch
        x = x.to(model.device)
        y = y.to(model.device)

        with torch.no_grad():
            preds = model(x)

        preds_np = preds.cpu().numpy().reshape(preds.shape[0], -1)
        y_np = y.cpu().numpy().reshape(y.shape[0], -1)

        for pred_sample, y_sample in zip(preds_np, y_np):
            mae = mean_absolute_error(y_sample, pred_sample)
            mae_losses.append(mae)
    mae_loss = np.mean(mae_losses)
    print(f'Mean Absolute Error: {mae_loss:.5f}')

    log_path = Path(logger.log_dir) / 'metrics.csv'
    df = pd.read_csv(log_path)
    df_train = df[['epoch', 'train_loss']].dropna()
    df_val = df[['epoch', 'val_loss']].dropna()
    df_merged = pd.merge(df_train, df_val, on='epoch')

    plt.plot(df_merged['epoch'], df_merged['train_loss'], label='Train Loss')
    plt.plot(df_merged['epoch'], df_merged['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training & Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    return mae_loss

In [ ]:
groups = [p.parts[4] for p in all_inputs]

gkf = GroupKFold(n_splits=Config.n_folds)
mae_scores = []
for fold, (tr_idx, val_idx) in enumerate(gkf.split(all_inputs, groups=groups)):
    mae_loss = run_train(fold, all_inputs, all_outputs, tr_idx, val_idx, Config)
    mae_scores.append(mae_loss)
print(f'     ----- {Config.n_folds}Folds Mean Absolute Error: {np.mean(mae_scores):.5f} -----')

In [ ]:
class TestDataset(Dataset):
    def __init__(self, test_files):
        self.test_files = test_files

    def __len__(self):
        return len(self.test_files)

    def __getitem__(self, i):
        test_file = self.test_files[i]
        x = np.load(test_file)
        x = apply_gaussian_filter(x)
        return torch.tensor(x, dtype=torch.float32), test_file.stem

In [ ]:
ds = TestDataset(test_files)
dl = DataLoader(ds, batch_size=Config.test_batch_size, num_workers=Config.num_workers, pin_memory=True)

In [ ]:
fold_preds = []
for fold in range(Config.n_folds):
    ckpt_path = glob.glob(f'model_fold{fold}/*.ckpt')[0]
    model = SeismicModel.load_from_checkpoint(ckpt_path)
    trainer = Trainer(accelerator='auto', devices=1 if torch.cuda.is_available() else None)
    pred = trainer.predict(model, dl)
    fold_preds.append(torch.cat(pred))
    del model
    gc.collect()
    torch.cuda.empty_cache()
avg_preds = torch.stack(fold_preds).mean(dim=0)

In [ ]:
x_cols = [f'x_{i}' for i in range(1, 70, 2)]
fieldnames = ['oid_ypos'] + x_cols

with open('submission.csv', 'wt', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for batch_preds, (_, oid_batch) in zip(avg_preds.split(Config.test_batch_size), dl):
        y_preds = batch_preds[:, 0].numpy()

        for y_pred, oid_test in zip(y_preds, oid_batch):
            for y_pos in range(70):
                row = dict(
                    zip(
                        x_cols,
                        [y_pred[y_pos, x_pos] for x_pos in range(1, 70, 2)]
                    )
                )
                row['oid_ypos'] = f"{oid_test}_y_{y_pos}"
                writer.writerow(row)